In [1]:
import pandas as pd
import numpy as np

Implementing Elo ratings to rank teams and predict outcomes is a powerful way to analyze team performance. Elo ratings are widely used in sports to evaluate the relative skill levels of teams and can be applied to simulate future games and predict playoff probabilities.

### Steps to Implement Elo Ratings:

1. **Initialize Elo Ratings:** Assign a starting Elo rating to each team. Typically, all teams start with a baseline rating, like 1500.

2. **Calculate Elo Ratings for Each Game:**
   - **Expected Score:** Calculate the expected score for each team based on their Elo ratings.
   - **Actual Score:** Determine the actual result of the game (win, loss, draw).
   - **Update Elo Ratings:** Adjust the Elo ratings based on the game outcome using the Elo formula.

3. **Simulate Future Games:** Use the updated Elo ratings to simulate future games, predicting outcomes and potential playoff scenarios.

### Explanation:

1. **Initialize Elo Ratings:**
   - All teams start with a baseline Elo rating of 1500.

2. **Calculate Elo Ratings for Each Game:**
   - **Expected Score:** The expected score for each team is calculated based on their current Elo ratings.
   - **Actual Score:** The actual game result is determined.
   - **Update Elo Ratings:** The Elo ratings are adjusted based on the difference between the actual and expected scores.

3. **Output Final Elo Ratings:**
   - After processing all games, the final Elo ratings for each team are displayed.


In [ ]:
df = pd.read_csv('Games.csv')

# Filter for recent seasons if needed (e.g., 2019-2023)
df_recent = df[df['season'].isin([2019, 2020, 2021, 2022, 2023])].copy()

# Initialize Elo ratings
initial_elo = 1500
elo_ratings = {team: initial_elo for team in pd.concat([df_recent['home_team'], df_recent['away_team']]).unique()}

# Elo parameters
K = 20  # K-factor, determines the sensitivity of Elo rating updates

# Function to calculate Elo ratings
def calculate_elo(rating1, rating2):
    expected_score1 = 1 / (1 + 10 ** ((rating2 - rating1) / 400))
    expected_score2 = 1 - expected_score1
    return expected_score1, expected_score2

# Process each game in chronological order
for i, row in df_recent.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Current Elo ratings
    home_elo = elo_ratings[home_team]
    away_elo = elo_ratings[away_team]
    
    # Calculate expected scores
    expected_home, expected_away = calculate_elo(home_elo, away_elo)
    
    # Determine actual scores
    if row['home_score'] > row['away_score']:
        actual_home = 1
        actual_away = 0
    elif row['home_score'] < row['away_score']:
        actual_home = 0
        actual_away = 1
    else:
        actual_home = actual_away = 0.5  # In case of a draw
    
    # Update Elo ratings
    elo_ratings[home_team] += K * (actual_home - expected_home)
    elo_ratings[away_team] += K * (actual_away - expected_away)
    
    # Store updated ratings
    df_recent.at[i, 'home_elo'] = elo_ratings[home_team]
    df_recent.at[i, 'away_elo'] = elo_ratings[away_team]

# Display the updated Elo ratings after processing all games
final_elo_ratings = pd.DataFrame.from_dict(elo_ratings, orient='index', columns=['Final Elo Rating']).sort_values(by='Final Elo Rating', ascending=False)
print(final_elo_ratings)


### Simulating Future Games:

To simulate future games using Elo ratings:
- Use the `calculate_elo` function to predict the outcome of upcoming games.
- Adjust the Elo ratings based on the simulated results to predict how the rest of the season might unfold.

### Advanced Usage:
- **Playoff Probabilities:** Simulate the entire season multiple times to estimate playoff probabilities based on the final Elo ratings.
- **Custom K-factor:** Adjust the `K` factor to make the Elo system more or less sensitive to game outcomes.
